<a href="https://colab.research.google.com/github/Ninlawat-Ph/sentiment-analysis/blob/master/Get_data_siriraj_piyamaharajkarun_hospital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install googletrans
!pip install tqdm --upgrade
!pip install twython

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.45.0)


## Import Modules

In [0]:
# web scraping
import requests
from bs4 import BeautifulSoup
from time import time, sleep
from random import randint

# Translation
from googletrans import Translator

# Utilities
from tqdm import tqdm
import numpy as np
import pandas as pd

# NLP
from nltk import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import mark_negation

In [0]:
# Download resources
import nltk

nltk.download("punkt")
nltk.download("vader_lexicon")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
def honestdoc_comment(url):
    """
    This function is to scrap data from a webboard (https://www.honestdocs.com).

    INPUT
    url : String
      URL of the target website
    
    OUTPUT
    comment : List
      List of comments
    score : List
      List of rating score
    """
    #create connection
    data = requests.get(url)
    print("requests code : {}".format(data.status_code)) 
    print("note\n2xx: success\n4xx, 5xx: error")
    
    #scrape comment and score
    start_time = time() #start scraping data from page1
    r = requests.get(url, params=dict(query="web scraping",page=1)) 
    soup = BeautifulSoup(r.text,"html.parser")
    n = len(soup.find_all("div",{"class":"comments__content"})) #find n of items in the page
    
    #extract each item
    comment = [soup.find_all("div",
                             {"class":"comments__content"})[i].get_text().strip() for i in range(0,n)]
    score = [soup.find_all("span",
                           {"class":"stars star-rating"})[i].attrs["data-score"] for i in range(0,n)]
    elapsed_time = time() - start_time #finish scraping data from page1
    print("Time used for scraping data from page - 1 : {} s".format(elapsed_time))
    sleep(randint(1,3)) #mimic human behavior
           
    p = 2 #start scraping data from page2
    while n > 0: #until the number of items in a page = 0
        start_time = time() 
        r = requests.get(url, params=dict(query="web scraping",page=p))
        soup = BeautifulSoup(r.text,"html.parser")
        n = len(soup.find_all("div",{"class":"comments__content"}))
        [comment.append(soup.find_all("div",
                                      {"class":"comments__content"})[i].get_text().strip()) for i in range(0,n)]
        [score.append(soup.find_all("span",
                                    {"class":"stars star-rating"})[i].attrs["data-score"]) for i in range(0,n)]
        elapsed_time = time() - start_time
        print("Time used for scraping data from page - {} : {} s".format(p, elapsed_time))
        p +=1
        sleep(randint(1,3))
    
    #backup data 
    pd.DataFrame({"comment": comment, 
                  "score": score}).to_csv("comment_"+str(url[url.rfind("/")+1:]) + ".csv", index=False)
    
    return comment, score

## Get data

In [0]:
nts, scores = honestdoc_comment(r"https://www.honestdocs.co/hospitals/siriraj-piyamaharajkarun-hospital")

requests code : 200
note
2xx: success
4xx, 5xx: error
Time used for scraping data from page - 1 : 0.9071540832519531 s
Time used for scraping data from page - 2 : 0.9052798748016357 s
Time used for scraping data from page - 3 : 0.8999674320220947 s
Time used for scraping data from page - 4 : 0.9121425151824951 s
Time used for scraping data from page - 5 : 0.8934996128082275 s
Time used for scraping data from page - 6 : 0.8986654281616211 s
Time used for scraping data from page - 7 : 0.9088301658630371 s
Time used for scraping data from page - 8 : 0.995145320892334 s
Time used for scraping data from page - 9 : 0.9213540554046631 s
Time used for scraping data from page - 10 : 0.8936314582824707 s
Time used for scraping data from page - 11 : 0.8948814868927002 s
Time used for scraping data from page - 12 : 0.8993244171142578 s
Time used for scraping data from page - 13 : 0.9049334526062012 s
Time used for scraping data from page - 14 : 0.8996050357818604 s
Time used for scraping data from

## Comment Translation The comments in this study are either Thai or English. Therefore, they should be standardized as English. Google translate API is the tool in this study AND Remove emoji in string

In [0]:
# restore data
data_comment_siriraj_piyamaharajkarun = pd.read_csv("comment_siriraj-piyamaharajkarun-hospital.csv")

### Remove emoji

In [0]:
#transform emoji to sting
import re
import sys
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [0]:
tqdm.pandas()
data_comment_siriraj_piyamaharajkarun["non_emojis"] = data_comment_siriraj_piyamaharajkarun.progress_apply(lambda x: remove_emoji(x["comment"]), axis=1)

100%|██████████| 396/396 [00:00<00:00, 16667.28it/s]


### Translate en to thai

In [0]:
def th2en(comment):
  return Translator().translate(comment, src="th", dest="en").text

In [0]:
data_comment_siriraj_piyamaharajkarun["en"] = data_comment_siriraj_piyamaharajkarun.progress_apply(lambda x: th2en(x["non_emojis"]), axis=1)

100%|██████████| 396/396 [00:47<00:00,  8.33it/s]


In [0]:
data_comment_siriraj_piyamaharajkarun.head()

,comment,score,non_emojis,en
0,วันนี้ไปเยี่ยมเพื่อนมาคะพึ่งคลอดลูก เป็นห้องรว...,5,วันนี้ไปเยี่ยมเพื่อนมาคะพึ่งคลอดลูก เป็นห้องรว...,Today I visited my friend's baby. But as well ...
1,ผมเคยปัสสาวะบ่อยมากๆ วันนึง 20รอบ \r\n แล้วได้...,4,ผมเคยปัสสาวะบ่อยมากๆ วันนึง 20รอบ \r\n แล้วได้...,I urinate very often around day 20.\r\n He was...
2,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,5,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,"Good Ceaa had discomfort, fever, doctor's parl..."
3,แม่ผมเข้ารับการรักษาอาการเส้นเลือดสมองตีบและเป...,1,แม่ผมเข้ารับการรักษาอาการเส้นเลือดสมองตีบและเป...,I've been treating cerebral thrombosis and pat...
4,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,5,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,"Good Ceaa had discomfort, fever, doctor's parl..."


### Restore data to csv file

In [0]:
data_comment_siriraj_piyamaharajkarun.to_csv("data_siriraj_piyamaharajkarun_en.csv", index=False)